In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core import display as ICD
import seaborn as sns
import glob
import scipy
import os
pd.set_option('display.max_columns', 100)
from helpers import *

In [2]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [3]:
DATA_FOLDER = './'

In [4]:
os.listdir(DATA_FOLDER)

['neural_network.ipynb',
 'regression_mat_year.csv',
 'ridge_regression.ipynb',
 '.ipynb_checkpoints',
 'feature_selection_rf.ipynb',
 'feature_selection_stepwise.ipynb']

### Regression matrix manipulation

Importing regression matrix

In [48]:
tot_df=pd.read_csv(DATA_FOLDER+'regression_mat_year.csv',index_col=0)

Transform absolute value and direction in vector components

In [49]:
tot_df=vectorize_wind_speed(tot_df)

In [9]:
tot_df.shape

(535470, 23)

Shorten the matrix for developping purposes

In [12]:
tot_df_small=tot_df
tot_df_small.shape

(535470, 23)

### Splitting Data pandas

In [14]:
tot_df_small_tr, tot_df_small_te=split_data(tot_df_small,0.3)

In [15]:
tot_df_tr, tot_df_te=split_data(tot_df,0.3)

### Standardize the features pandas

In [16]:
# for trying small dataset and debugging purposes
X_small_tr_standard=(tot_df_small_tr.drop(columns=['u_x', 'u_y','u_z'])-tot_df_small_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/tot_df_small_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [17]:
X_tr_standard=(tot_df_tr.drop(columns=['u_x', 'u_y','u_z'])-tot_df_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/tot_df_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [18]:
y_tr_small=tot_df_small_tr[['u_x','u_y']]

In [19]:
y_tr=tot_df_tr[['u_x','u_y']]

In [20]:
# for trying small dataset and debugging purposes
X_small_te_standard=(tot_df_small_te.drop(columns=['u_x', 'u_y','u_z'])-tot_df_small_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/tot_df_small_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [21]:
X_te_standard=(tot_df_te.drop(columns=['u_x', 'u_y','u_z'])-tot_df_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/tot_df_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [22]:
y_te_small=tot_df_small_te[['u_x','u_y']]

In [23]:
y_te=tot_df_te[['u_x','u_y']]

# Stepwise feature selection for one year

A method to define the most important number of features is backward selection: in this case at the beginning all the features are considered to be important. From a physical point of view, however, it is clear that the free stream wind speed `u_top_x`, `u_top_y` and `h` are going to be important. As a consequence forward feature selection will be implemented. The null hypothesis is that the true regression matrix is only composed by the three columns chosen and at every step a column will be added if the improvement in the value of $R^2$ is more unlikly than the significant level chosen. Then Bidirectional elimination is also implemented: consequently a backward step will be implemented to check if the new column added has a strong covariance with one the previous added columns. In this way the order to addition of the columns won't matter.

In [24]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.005, 
                       threshold_out = 0.1, 
                       verbose=True):
    
    # the threshold in is specified at 0.01 confidence level for the output meaning that for each\
    # velocity component is going to be 0.005 using bonferroni method
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        
        # make a new series with a set of index being the columns you start with
        new_pval = pd.Series(index=excluded)
        
        for new_column in excluded:
            # make the model by adding a column. N.B the order to addition of columns doesn't matter since 
            # all parameters are added one by one and then the most unlikely is selected on the base that the
            # hypothesis of having the estimators being B=0
            
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            # the parameter used to calculate a two tail relative significance level can be obtained
            #based on the R square value of the regression and the 
            
            new_pval[new_column] = model.pvalues[new_column]
        
        # add the column which produced the most unluckily p value
        best_pval = new_pval.min()
        
        # here it starts the backward step. this is useful to contract the order of selection of the features
        # the order of selection matters so the features will only be kept if their removal is significant
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

Perform the stepwise feature selection for the horizontal velocity of u

In [25]:
result = stepwise_selection(X_small_tr_standard, y_tr_small.iloc[:,0], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  East temperature [°C]          with p-value 0.0
Add  Net Solar radiation [W/m$^2$]  with p-value 0.0
Add  Net Far Infrared radiation [W/m$^2$] with p-value 8.08656e-114
Add  u_top_z                        with p-value 1.93059e-104
Add  sonic_temp                     with p-value 2.21019e-56
Add  South temperature [°C]         with p-value 6.98168e-185
Drop East temperature [°C]          with p-value 0.344227


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  North temperature [°C]         with p-value 2.44192e-32
Add  West temperature [°C]          with p-value 6.31725e-36
Add  Pyranometer Lower Irradiance [W/m$^2$] with p-value 3.01788e-24
Add  sonic_temp_top                 with p-value 1.0122e-10
Add  Sensor Ground temperature [°C] with p-value 1.1434e-08
resulting features:
['u_top_y', 'u_top_x', 'h', 'Net Solar radiation [W/m$^2$]', 'Net Far Infrared radiation [W/m$^2$]', 'u_top_z', 'sonic_temp', 'South temperature [°C]', 'North temperature [°C]', 'West temperature [°C]', 'Pyranometer Lower Irradiance [W/m$^2$]', 'sonic_temp_top', 'Sensor Ground temperature [°C]']


Perform the stepwise feature selection for the vertical velocity of u

In [26]:
result = stepwise_selection(X_small_tr_standard, y_tr_small.iloc[:,1], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  u_top_z                        with p-value 2.15848e-33
Add  Net Far Infrared radiation [W/m$^2$] with p-value 2.63548e-33
Add  Net (total) radiation [W/m$^2$] with p-value 3.71178e-59
Add  Sensor Ground temperature [°C] with p-value 1.22336e-28
Add  North temperature [°C]         with p-value 1.7022e-23
Add  Sky temperature [°C]           with p-value 3.39554e-44
Drop Sensor Ground temperature [°C] with p-value 0.210017


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  Pyranometer Lower Irradiance [W/m$^2$] with p-value 2.96997e-30
Add  East temperature [°C]          with p-value 2.04665e-06
Add  sonic_temp                     with p-value 4.59873e-06
Add  sonic_temp_top                 with p-value 3.13838e-77
Drop h                              with p-value 0.245042
Add  West temperature [°C]          with p-value 1.29486e-06
Add  Radiometer Ground temperature [°C] with p-value 0.000103304
Drop Net Far Infrared radiation [W/m$^2$] with p-value 0.292728
Add  South temperature [°C]         with p-value 0.000134724
resulting features:
['u_top_y', 'u_top_x', 'u_top_z', 'Net (total) radiation [W/m$^2$]', 'North temperature [°C]', 'Sky temperature [°C]', 'Pyranometer Lower Irradiance [W/m$^2$]', 'East temperature [°C]', 'sonic_temp', 'sonic_temp_top', 'West temperature [°C]', 'Radiometer Ground temperature [°C]', 'South temperature [°C]']


In [ ]:
tot_df_small_standard.columns.values

# Stepwise feature selection for each season

In [28]:
df_spring, df_summer, df_autumn, df_winter=season_splitter(tot_df)

In [29]:
df_spring_tr, df_spring_te=split_data(df_spring,0.3)
df_summer_tr, df_summer_te=split_data(df_summer,0.3)
df_autumn_tr, df_autumn_te=split_data(df_autumn,0.3)
df_winter_tr, df_winter_te=split_data(df_winter,0.3)

In [41]:
X_spring_tr_standard=(df_spring_tr.drop(columns=['u_x', 'u_y','u_z'])-df_spring_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_spring_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_autumn_tr_standard=(df_autumn_tr.drop(columns=['u_x', 'u_y','u_z'])-df_autumn_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_autumn_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_summer_tr_standard=(df_summer_tr.drop(columns=['u_x', 'u_y','u_z'])-df_summer_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_summer_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_winter_tr_standard=(df_winter_tr.drop(columns=['u_x', 'u_y','u_z'])-df_winter_tr.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_winter_tr.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [31]:
y_spring_tr=df_spring_tr[['u_x','u_y']]
y_summer_tr=df_summer_tr[['u_x','u_y']]
y_autumn_tr=df_autumn_tr[['u_x','u_y']]
y_winter_tr=df_winter_tr[['u_x','u_y']]

In [39]:
X_spring_te_standard=(df_spring_te.drop(columns=['u_x', 'u_y','u_z'])-df_spring_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_spring_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_autumn_te_standard=(df_autumn_te.drop(columns=['u_x', 'u_y','u_z'])-df_autumn_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_autumn_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_summer_te_standard=(df_summer_te.drop(columns=['u_x', 'u_y','u_z'])-df_summer_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_summer_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)
X_winter_te_standard=(df_winter_te.drop(columns=['u_x', 'u_y','u_z'])-df_winter_te.drop(columns=['u_x', 'u_y','u_z']).mean(axis=0))/df_winter_te.drop(columns=['u_x', 'u_y','u_z']).std(axis=0)

In [33]:
y_spring_te=df_spring_te[['u_x','u_y']]
y_summer_te=df_summer_te[['u_x','u_y']]
y_autumn_te=df_autumn_te[['u_x','u_y']]
y_winter_te=df_winter_te[['u_x','u_y']]

### Spring

Perform the stepwise feature selection for the horizontal velocity of u

In [34]:
result = stepwise_selection(X_spring_tr_standard, y_spring_tr.iloc[:,0], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Net Solar radiation [W/m$^2$]  with p-value 0.0
Add  u_top_z                        with p-value 2.36356e-46
Add  Net Far Infrared radiation [W/m$^2$] with p-value 6.03955e-16
Add  sonic_temp                     with p-value 2.5763e-09
Add  Radiometer Ground temperature [°C] with p-value 1.45747e-69
Drop Net Far Infrared radiation [W/m$^2$] with p-value 0.539045


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  West temperature [°C]          with p-value 5.82356e-44
Add  East temperature [°C]          with p-value 6.20289e-10
Add  South temperature [°C]         with p-value 2.23759e-16
Drop Net Solar radiation [W/m$^2$]  with p-value 0.460525
Add  sonic_temp_top                 with p-value 0.000147081
Add  Sensor Ground temperature [°C] with p-value 8.48581e-05
Add  North temperature [°C]         with p-value 1.92986e-07
resulting features:
['u_top_y', 'u_top_x', 'h', 'u_top_z', 'sonic_temp', 'Radiometer Ground temperature [°C]', 'West temperature [°C]', 'East temperature [°C]', 'South temperature [°C]', 'sonic_temp_top', 'Sensor Ground temperature [°C]', 'North temperature [°C]']


Perform the stepwise feature selection for the vertical velocity of u

In [35]:
result = stepwise_selection(X_spring_tr_standard, y_spring_tr.iloc[:,1], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Sky temperature [°C]           with p-value 1.64183e-08
Add  sonic_temp_top                 with p-value 4.11106e-14
Add  Net Far Infrared radiation [W/m$^2$] with p-value 8.65484e-11
Add  North temperature [°C]         with p-value 4.63508e-06
Add  Pyranometer Lower Irradiance [W/m$^2$] with p-value 3.00878e-05
Add  Net (total) radiation [W/m$^2$] with p-value 2.86421e-06
Add  East temperature [°C]          with p-value 0.000845914
Drop sonic_temp_top                 with p-value 0.296544


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  Pyrgeometer Lower Irradiance [W/m$^2$] with p-value 0.00116689
Add  Radiometer Ground temperature [°C] with p-value 0.00259041
resulting features:
['u_top_y', 'u_top_x', 'h', 'Sky temperature [°C]', 'Net Far Infrared radiation [W/m$^2$]', 'North temperature [°C]', 'Pyranometer Lower Irradiance [W/m$^2$]', 'Net (total) radiation [W/m$^2$]', 'East temperature [°C]', 'Pyrgeometer Lower Irradiance [W/m$^2$]', 'Radiometer Ground temperature [°C]']


### Summer

Perform the stepwise feature selection for the vertical velocity of u

In [36]:
result = stepwise_selection(X_summer_tr_standard, y_summer_tr.iloc[:,0], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  Net Solar radiation [W/m$^2$]  with p-value 0.0
Drop h                              with p-value 0.562422
Add  u_top_z                        with p-value 2.14012e-49
Add  Net (total) radiation [W/m$^2$] with p-value 1.67741e-37
Add  Pyranometer Upper Irradiance [W/m$^2$] with p-value 1.41071e-05
Add  sonic_temp                     with p-value 4.99485e-08
Add  South temperature [°C]         with p-value 5.72628e-59
Drop Pyranometer Upper Irradiance [W/m$^2$] with p-value 0.124046
Add  East temperature [°C]          with p-value 4.00664e-10
Add  sonic_temp_top                 with p-value 1.84885e-07
Add  h                              with p-value 1.13923e-10
Add  Radiometer Ground temperature [°C] with p-value 0.000366695
Add  Sky temperature [°C]           with p-value 8.21858e-07
Add  Pyrgeometer Upper Irradiance [W/m$^2$] with p-value 2.44039e-07
Add  West temperature [°C]          with p-value 0.00034787
Add  Pyranometer Lower Irradiance [W/m$^2$] with p-value 0.00119032
Add

Perform the stepwise feature selection for the vertical velocity of u

In [37]:
result = stepwise_selection(X_summer_tr_standard, y_summer_tr.iloc[:,1], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Pyrgeometer Upper Irradiance [W/m$^2$] with p-value 3.00676e-63
Add  Pyranometer Upper Irradiance [W/m$^2$] with p-value 1.63034e-06
Add  sonic_temp                     with p-value 0.00388025
Add  sonic_temp_top                 with p-value 5.51518e-153
Add  u_top_z                        with p-value 7.03443e-08
resulting features:
['u_top_y', 'u_top_x', 'h', 'Pyrgeometer Upper Irradiance [W/m$^2$]', 'Pyranometer Upper Irradiance [W/m$^2$]', 'sonic_temp', 'sonic_temp_top', 'u_top_z']


### Winter

Perform the stepwise feature selection for the vertical velocity of u

In [42]:
result = stepwise_selection(X_winter_tr_standard, y_winter_tr.iloc[:,0], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Net Solar radiation [W/m$^2$]  with p-value 3.55438e-58
Add  sonic_temp                     with p-value 5.92245e-39
Add  Pyrgeometer Lower Irradiance [W/m$^2$] with p-value 1.5673e-45
Add  sonic_temp_top                 with p-value 7.31263e-06
Drop Net Solar radiation [W/m$^2$]  with p-value 0.208103


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  East temperature [°C]          with p-value 0.000269281
Add  North temperature [°C]         with p-value 0.00134734
Add  Sensor Ground temperature [°C] with p-value 0.00459355
resulting features:
['u_top_y', 'u_top_x', 'h', 'sonic_temp', 'Pyrgeometer Lower Irradiance [W/m$^2$]', 'sonic_temp_top', 'East temperature [°C]', 'North temperature [°C]', 'Sensor Ground temperature [°C]']


Perform the stepwise feature selection for the vertical velocity of u

In [43]:
result = stepwise_selection(X_winter_tr_standard, y_winter_tr.iloc[:,1], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  u_top_z                        with p-value 6.73822e-26
Add  sonic_temp                     with p-value 1.22286e-26
Add  Pyrgeometer Upper Irradiance [W/m$^2$] with p-value 5.76596e-05
Add  Sky temperature [°C]           with p-value 6.27406e-19
resulting features:
['u_top_y', 'u_top_x', 'h', 'u_top_z', 'sonic_temp', 'Pyrgeometer Upper Irradiance [W/m$^2$]', 'Sky temperature [°C]']


### Autumn

Perform the stepwise feature selection for the vertical velocity of u

In [44]:
result = stepwise_selection(X_autumn_tr_standard, y_autumn_tr.iloc[:,0], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Pyranometer Lower Irradiance [W/m$^2$] with p-value 1.2576e-224
Add  u_top_z                        with p-value 7.10814e-78
Add  Pyrgeometer Upper Irradiance [W/m$^2$] with p-value 9.89929e-35
Add  Net Far Infrared radiation [W/m$^2$] with p-value 1.14009e-14
Add  sonic_temp_top                 with p-value 5.42044e-70
Add  South temperature [°C]         with p-value 5.42091e-62
Drop Pyranometer Lower Irradiance [W/m$^2$] with p-value 0.268612


/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:62: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.


Add  North temperature [°C]         with p-value 4.51031e-20
Add  sonic_temp                     with p-value 4.52847e-12
Add  East temperature [°C]          with p-value 1.21693e-06
Add  Net (total) radiation [W/m$^2$] with p-value 3.19671e-11
Add  Sensor Ground temperature [°C] with p-value 2.09764e-11
Add  Radiometer Ground temperature [°C] with p-value 8.3854e-06
Add  West temperature [°C]          with p-value 0.000892906
resulting features:
['u_top_y', 'u_top_x', 'h', 'u_top_z', 'Pyrgeometer Upper Irradiance [W/m$^2$]', 'Net Far Infrared radiation [W/m$^2$]', 'sonic_temp_top', 'South temperature [°C]', 'North temperature [°C]', 'sonic_temp', 'East temperature [°C]', 'Net (total) radiation [W/m$^2$]', 'Sensor Ground temperature [°C]', 'Radiometer Ground temperature [°C]', 'West temperature [°C]']


Perform the stepwise feature selection for the vertical velocity of u

In [45]:
result = stepwise_selection(X_autumn_tr_standard, y_autumn_tr.iloc[:,1], initial_list=['u_top_y', 'u_top_x','h'])

print('resulting features:')
print(result)

/Users/gianlucamancini/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:49: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  Pyrgeometer Upper Irradiance [W/m$^2$] with p-value 3.82433e-36
Add  Net (total) radiation [W/m$^2$] with p-value 1.14854e-14
Add  North temperature [°C]         with p-value 5.55712e-14
Add  Pyranometer Upper Irradiance [W/m$^2$] with p-value 3.50709e-36
Add  East temperature [°C]          with p-value 1.73524e-11
Add  South temperature [°C]         with p-value 5.78652e-11
Add  Sky temperature [°C]           with p-value 6.12514e-07
Add  u_top_z                        with p-value 8.12656e-07
resulting features:
['u_top_y', 'u_top_x', 'h', 'Pyrgeometer Upper Irradiance [W/m$^2$]', 'Net (total) radiation [W/m$^2$]', 'North temperature [°C]', 'Pyranometer Upper Irradiance [W/m$^2$]', 'East temperature [°C]', 'South temperature [°C]', 'Sky temperature [°C]', 'u_top_z']


In [ ]:
def forward_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.005, 
                       threshold_out = 0.1, 
                       verbose=True):
    
    # the threshold in is specified at 0.01 confidence level for the output meaning that for each\
    # velocity component is going to be 0.005 using bonferroni method
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        
        # make a new series with a set of index being the columns you start with
        new_pval = pd.Series(index=excluded)
        
        for new_column in excluded:
            # make the model by adding a column. N.B the order to addition of columns doesn't matter since 
            # all parameters are added one by one and then the most unlikely is selected on the base that the
            # hypothesis of having the estimators being B=0
            
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            # the parameter used to calculate f distribution is the R square value of the regression and the 
            # relative significance level
            new_pval[new_column] = model.f_pvalue
        
        # add the column which produced the most unluckily p value
        best_pval = new_pval.min()
        
        # here it starts the backward step. this is useful to contract the order of selection of the features
        # the order of selection matters so the features will only be kept if their removal is significant
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

#         # backward step
#         model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
#         # use all coefs except intercept
#         pvalues = model.pvalues.iloc[1:]
#         worst_pval = pvalues.max() # null if pvalues is empty
#         if worst_pval > threshold_out:
#             changed=True
#             worst_feature = pvalues.argmax()
#             included.remove(worst_feature)
#             if verbose:
#                 print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
#         if not changed:
#             break
    return included

# Backward feature selection

In [ ]:
alphas=np.logspace(-1,10,100)

#leave one out cv method is used
estimator = RidgeCV(alphas, cv=None, store_cv_values=True)
selector=RFECV(estimator, min_features_to_select=3, step=1, verbose=False, cv=5)
selector = selector.fit(X_tr_scaled_small, y_tr_small[:,0].reshape(-1, 1))
selector.support_

In [ ]:
alphas=np.logspace(-1,10,100)

#leave one out cv method is used
estimator = RidgeCV(alphas, cv=None, store_cv_values=True)
selector=RFECV(estimator, min_features_to_select=3, step=1, verbose=False, cv=5)
selector = selector.fit(X_tr_scaled_small, y_tr_small[:,1].reshape(-1, 1))
selector.support_

In [ ]:
len(tot_df_small_standard.drop(columns=['u_x', 'u_y','u_z']).columns.values)
